# Closest Pair of Points

# Import packages

In [56]:
import numpy as np
import sys
import math
import time
import os
from os import listdir
from os.path import isfile, join
import random

# Calculate the distance between two points

In [57]:
def dist(v1,v2):   
    if v1[0] is None or v1[1] is None or v2[0] is None or v2[1] is None:
        print("Error: some x or y coordinate is null.")
        return float("inf") 
    if v1[0]-v2[0] == 0 and v1[1]-v2[1] == 0: return 0
    s=float((abs(v1[0]-v2[0]))**2 + abs((v1[1]-v2[1]))**2)
    return math.sqrt(s)  

# Remove duplicate points

In [138]:
def noDup(P):
    seen = []
    for pt in P:
        if pt not in seen:
            seen.append(pt)
    return seen

# Find min distance and closest pair among points inside strip

In [58]:
def stripP(P, minD, minCP):
    n = len(P)
    for i in range(0, n):
        k = 1
        while i+k < n and P[i+k][1] - P[i][1] < minD:
            d = dist(P[i], P[i+k])
            if d < minD:
                minD = d
                minCP = P[i], P[i+k]
            k += 1
    return minD, minCP

# Generate uniformly random points in unit square output files

In [59]:
def genUnif(numF, increN):   
    for i in range(numF): 
        testU = './Test/randTest/uni'
        testU += str(i)
        testU += '.txt'              
        try:
            os.remove(testU)
        except OSError:
            pass
        
        testUOut = open(testU,'a')
        
        for j in range((i+1)*increN):
            px = random.uniform(0, 1)
            py = random.uniform(0, 1)
            testUOut.write(str(px))
            testUOut.write("\t")
            testUOut.write(str(py))
            testUOut.write("\n")
        testUOut.close()

# Generate points between (0,0) and (0,1) output files

In [60]:
def genVer(numF, increN):    
    for i in range(numF): 
        outPath = './Test/verTest/ver'  
        outPath += str(i)
        outPath += '.txt'  
        try:
            os.remove(outPath)
        except OSError:
            pass
        
        outF = open(outPath,'a')

        for j in range((i+1)*increN):     
            py = float(j/((i+1)*increN))
            outF.write(str(0))
            outF.write("\t")
            outF.write(str(py))
            outF.write("\n")
        outF.close()

# Test on a directory of tests

In [61]:
def dirTest(path, outN, test): 
    os.chdir(path)
    fileSet = [f for f in listdir(path) if isfile(join(path, f)) and f.startswith(test)]    
    try:
        os.remove(outN)
    except OSError:
        pass

    # output report to a file
    out = open(outN,'a')
    # write a header
    out.write("num")
    out.write("\t")
    out.write("v1")
    out.write("\t")
    out.write("v2")
    out.write("\t")
    out.write("v3")
    out.write("\t")
    out.write("\n")
    out.close()

    for i in fileSet:
        P = []
        with open(i) as f:
            P.clear()
            count = 0
            PX, PY = 0, 0
            for line in f:
                data = line.split()

                for p in data:
                    if count % 2 == 0: 
                        PX = float(p)
                    else: 
                        PY = float(p)
                        P.append([PX, PY])
                    count += 1

        out = open(outN,'a')
        out.write(str(count/2))
        out.write('\t')
        resultSet = [CP1(P), CP2(P), CP3(P)]
        for result in resultSet:
            s = result[0]
            out.write(str(s))
            out.write('\t')    
        out.write('\n')
        out.close()

# Test on a single file

In [62]:
def fileTest(fileN, outN):

    with open(fileN) as f:  
        # create a new output file
        try:
            os.remove(outN)
        except OSError:
            pass

        P = []
        P.clear()
        count = 0
        PX, PY = 0, 0
        for line in f:
            data = line.split()

            for p in data:
                if count % 2 == 0: 
                    PX = float(p)
                else: 
                    PY = float(p)
                    P.append([PX, PY])
                count += 1
        n = str(count/2)              
        resultSet = [CP1(P), CP2(P), CP3(P)]
        count = 0
        for result in resultSet:
            count += 1
            t = result[0]
            d = result[1]
            pair = []
            if d != float("inf"): pair = result[2]

            # output report to a file   
            out = open(outN,'a')
            out.write("Version ")
            out.write(str(count))
            out.write(":\nNumber of pt: ")
            out.write(str(n))
            out.write("\n")
            out.write("closest pair: ") 
            out.write(str(pair))
            out.write("\ndistance btw: ")
            out.write(str(d))
            out.write("\ntime: ")
            out.write(str(t))
            out.write("s\n")
            if (d == float("inf")): out.write("Error: only one point, cannot find closest pair.")
            out.write("\n\n")

        out.close()

# Version 1 $\Theta(n^2)$

In [94]:
def brute(P):
    n=len(P)
    minD = float("inf")    
    cpMin=[]
    if n >= 2:
        for i in range(0, n):
            for j in range(i+1, n):
                d = dist(P[i], P[j])
                pt = [P[i], P[j]]
                if d < minD:
                    minD = d
                    cpMin.clear()
                    cpMin.append(pt)      
    return minD, cpMin

In [142]:
def CP1(P):
    start = time.time()    
    P = noDup(P)
    n=len(P)
    if n == 1: 
        print("Error: only one point, cannot find closest pair.")
        return time.time()-start,float("inf"),[]
    
    result = brute(P)
    perfT=time.time()-start
    return perfT, result[0], result[1][0]

# Version 2 $\Theta(nlog^2n)$

In [140]:
def CPHelper2(P):
    minD=float("inf")
    minCP=[]
    n = len(P)
    if n <= 3: return brute(P)
    
    # divide points into two equal sized parts
    mid = n // 2 
    midX = P[mid][0]
    PL = P[:mid]
    PR = P[mid:]
    
    # find min distance of both sides
    DL, cpL = CPHelper2(PL)
    DR, cpR = CPHelper2(PR)
    minD, minCP = DL, cpL
    if DR < minD: minD, minCP = DR, cpR
    
    # build a strip of points within min distance of separation line    
    # update min distance and closest pair if find a smaller min distance
    sp = []
    for pt in P:
        if abs(pt[0]-midX) < minD: sp.append(pt)            
    # sort by y
    sp = sorted(sp, key=lambda k: k[1])     
    return stripP(sp, minD, minCP)
    
def CP2(P):
    start=time.time()
    P = noDup(P)
    if len(P) == 1: 
        print("Error: only one point, cannot find closest pair.")
        return time.time()-start,float("inf"),[]
    
    # pre-sort by x
    P = sorted(P, key=lambda k: k[0])
    result = CPHelper2(P)  
    perfT=time.time()-start
    return perfT, result[0], result[1]

# Version 3 $\Theta(nlogn)$

In [136]:
def CPHelper3(PX, PY):
    n = len(PX)
    if n <= 3: return brute(PX)
    
    minD=float("inf")
    minCP=[]  
    
    mid = n // 2
    XL = PX[:mid]
    XR = PX[mid:]
    YL,YR=[],[]
    midX=PX[mid][0]
    
    # divide points sort by y into two halves
    # the split line is at middle of x
    # assume x coordinates are distinct
    for p in PY:
        if p[0] < midX: YL.append(p)
        else: YR.append(p)
    
    DL, cpL = CPHelper3(XL, YL)
    DR, cpR = CPHelper3(XR, YR)
    minD, minCP = DL, cpL
    if DR < minD: minD, minCP = DR, cpR
                
    sp = []
    for pt in PY:
        if abs(pt[0]-midX) < minD: sp.append(pt)            

    return stripP(sp, minD, minCP)
    
def CP3(P):
    start=time.time()
    P = noDup(P)
    if len(P) == 1: 
        print("Error: only one point, cannot find closest pair.")
        return time.time()-start,float("inf"),[]
    
    # sort by x
    PX = sorted(P, key=lambda k: k[0])
    # sort by y
    PY = sorted(P, key=lambda k: k[1])
    result = CPHelper3(PX, PY)
    perfT=time.time()-start
    return perfT, result[0], result[1]

# Small test by hand input

In [139]:
# small test by hand input
v1=[0,0]
v2=[2,1]
v3=[1.5,22]
v4=[2.5,1]
v5=[3,1.2]
P1=[v1,v2,v3,v4,v5]

# V1 test
print("v1,",CP1(P1))

# V2 test
print("v2,", CP2(P1))

# V3 test
print("v3,", CP3(P1))

v1, (4.100799560546875e-05, 0.5, [[2, 1], [2.5, 1]])
v2, (3.695487976074219e-05, 0.5, [[[2, 1], [2.5, 1]]])
v3, (3.504753112792969e-05, 0.5, [[[2, 1], [2.5, 1]]])


# Large test with uniformly random input

In [143]:
#genUnif(100, 5)
path = './Test/randTest/'
outN = './outRand.csv'
test = 'uni'
dirTest(path, outN, test)

# Large test with points between (0,0) and (0,1)

In [132]:
#genVer(100,5)
path = './Test/verTest'
outN = './outVer.csv'
test = 'ver'
dirTest(path, outN, test)

# Sample turn-in tests

In [102]:
fileN1 = './Test/turninTest/points-test1.txt'
outN1 = './out-test1.txt'
fileN2 = './Test/turninTest/points-test2.txt'
outN2 = './out-test2.txt'
fileN3 = './Test/turninTest/points-test3.txt'
outN3 = './out-test3.txt'
fileTest(fileN1, outN1)
fileTest(fileN2, outN2)
fileTest(fileN3, outN3)

Error: only one point, cannot find closest pair.
Error: only one point, cannot find closest pair.
Error: only one point, cannot find closest pair.
